In [2]:
from datasets import load_dataset
import os
import torch

os.environ["HF_HOME"] = "/home/jeromeku/dev/third_party/torchtune/hf_cache"
dataset = load_dataset(
    "philschmid/dolly-15k-oai-style",
    split="train",
)

In [7]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
dtype = torch.bfloat16
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=dtype)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    processing_class=tokenizer,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=1,
        max_steps=10,
        logging_steps=1,
        output_dir="outputs",
        seed=3407,
        fp16=model.get_input_embeddings().weight.dtype == torch.float16,
        bf16=model.get_input_embeddings().weight.dtype == torch.bfloat16,
        report_to="wandb",  # For W&B
        dataset_num_proc=4,
    ),
)

Map (num_proc=4):   0%|          | 0/15011 [00:00<?, ? examples/s]

In [9]:
loader = trainer.get_train_dataloader()

In [10]:
batch = next(iter(loader))

In [12]:
batch["input_ids"].shape

torch.Size([4, 249])

In [15]:
print(tokenizer.decode(batch["input_ids"][0], skip_special_tokens=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 20 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a letter I can send to the company that installed my swimming pool. Explain to them that the pool has two leaks and that I'd like to make a warranty claim. Request a reply within the next 30 days.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Date: April 4, 2023
From: Firstname Lastname
To: California Pool Company

I purchased the home at 123 Main Street, Riverside, California in July 2022. The property has a pool that you installed in 2021. We have had multiple issues with the pool, including two leaks in the underground water lines supplying the automatic pool leveler. I understand the pool has a two-year warranty; I would like you to inspect the issues we have found before the warranty expires. I have enclosed photographs of the issues we have found so far. Please call me at 123-345-3883 to sch